<a href="https://colab.research.google.com/github/ATOMconsortium/AMPL/blob/master/atomsci/ddm/examples/tutorials/11_CHEMBL26_SCN5A_IC50_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Building a Graph Convolutional Network Model for Drug Response Prediction</h1>

The ATOM Modeling PipeLine (AMPL; https://github.com/ATOMconsortium/AMPL) is an open-source, modular, extensible software pipeline for building and sharing models to advance in silico drug discovery.


## Time to run: 6 minutes

In [1]:
!date

Tue Aug 10 23:45:33 UTC 2021


## Change your runtime to GPU 

Go to **Runtime** --> Change **runtime type** to "GPU"


## Goal: Use AMPL for predicting binding affinities -IC50 values- of ligands that could bind to human **Sodium channel protein type 5 subunit alpha** protein using Graph Convolutional Network Model. ChEMBL database is the source of the binding affinities (pIC50).

In this notebook, we describe the following steps using AMPL:

1.   Read a ML ready dataset
2.   Fit a Graph Convolutional model
3.   Predict pIC50 values of withheld compounds

## Set up
We first import the AMPL modules for use in this notebook.

The relevant AMPL modules for this example are listed below:

|module|Description|
|-|-|
|`atomsci.ddm.pipeline.model_pipeline`|The model pipeline module is used to fit models and load models for prediction.|
|`atomsci.ddm.pipeline.parameter_parser`|The parameter parser reads through pipeline options for the model pipeline.|
|`atomsci.ddm.utils.curate_data`|The curate data module is used for data loading and pre-processing.|
|`atomsci.ddm.utils.struct_utils`|The structure utilities module is used to process loaded structures.|


import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

## Install AMPL 

In [ ]:
! pip install rdkit-pypi
! pip install --pre deepchem # 2.6.0 dev 
# ! pip install deepchem==2.5.0

import deepchem
# print(deepchem.__version__)
! pip install umap
! pip install llvmlite==0.35.0  --ignore-installed
! pip install umap-learn
! pip install molvs
! pip install bravado

In [ ]:
import deepchem as dc

# get the Install AMPL_GPU_test.sh
!wget https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU_test.sh

# run the script to install AMPL
! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh

In [4]:
dc.__version__

'2.6.0.dev'

In [5]:
# We temporarily disable warnings for demonstration.
# FutureWarnings and DeprecationWarnings are present from some of the AMPL 
# dependency modules.
import warnings
warnings.filterwarnings('ignore')
import json
import requests
import sys

In [6]:
import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

2021-08-10 23:46:32,227 Model tracker client not supported in your environment; can look at models in filesystem only.


In [7]:
import os
os.mkdir('chembl_activity_models')

**Let us display the dataset**

In [8]:
import pandas as pd
import requests
import io
url = 'https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/datasets/ChEMBL26_SCN5A_IC50_human_ml_ready.csv'
download = requests.get(url).content
df = pd.read_csv(url, index_col=0)
# Reading the downloaded content and turning it into a pandas dataframe
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.iloc[0:5, 0:5]
df.to_csv('ChEMBL26_SCN5A_IC50_human_ml_ready.csv', index=False)

In [9]:
df

,base_rdkit_smiles,compound_id,pIC50,relation,active
0,O=S(=O)(Nc1nccs1)c1ccc2c(c1)OCCN2c1ccc(Cl)cc1O...,CHEMBL3948159,5.806875,NaN,1
1,Fc1cccc(Cn2ccc3c(OC4CCN(Cc5ccccn5)CC4)ncnc32)c1F,CHEMBL2012299,7.698970,NaN,1
2,COc1cc(-c2cccc(F)c2)c(Cl)cc1-c1ncc(O)c2cc(S(=O...,CHEMBL4089326,5.000000,<,0
3,O=C(Nc1cccc(C(F)(F)F)c1-c1ccccn1)c1cccc(-c2ncc...,CHEMBL4084926,5.721246,NaN,1
4,CC(C)COc1ncc(-c2nn(C)c3cc(C(=O)NS(C)(=O)=O)ccc...,CHEMBL4280298,5.186552,NaN,1
...,...,...,...,...,...
1724,O=C(NCc1ccc(C(F)(F)F)cc1)C1c2ccccc2C(=O)N1CCc1...,CHEMBL2164387,5.795880,NaN,1
1725,COc1ccc(Cl)c(-c2ccc(NC(=O)c3ccnn3C)nc2N)c1,CHEMBL3589904,4.431798,NaN,1
1726,N#Cc1cc(F)ccc1N1CCOc2cc(S(=O)(=O)Nc3nccs3)ccc21,CHEMBL3955931,5.000000,<,0
1727,Clc1cn(Cc2ccccc2)c2ncnc(OC3CCN(Cc4cscn4)CC3)c12,CHEMBL2012181,7.096910,NaN,1


base_splitter
Description:	Type of splitter to use for train/validation split if temporal split used for test set. May be random, scaffold, or ave_min. The allowable choices are set in splitter.py

Check here for details, https://github.com/ATOMconsortium/AMPL/blob/master/atomsci/ddm/docs/PARAMETERS.md

In [10]:
split_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"split_only": "True",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "False",
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [11]:
split_params = parse.wrapper(split_config)

In [12]:
split_model = mp.ModelPipeline(split_params)

In [13]:
split_uuid = split_model.split_dataset()
split_uuid

2021-08-10 23:46:32,814 Attempting to load featurized dataset
2021-08-10 23:46:32,832 Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
2021-08-10 23:46:32,833 Featurized dataset not previously saved for dataset ChEMBL26_SCN5A_IC50_human_ml_ready, creating new
2021-08-10 23:46:32,849 Featurizing sample 0
2021-08-10 23:46:40,250 Featurizing sample 1000


number of features: 75


2021-08-10 23:46:45,925 Splitting data by scaffold
2021-08-10 23:46:57,004 Dataset split table saved to /content/ChEMBL26_SCN5A_IC50_human_ml_ready_train_valid_test_scaffold_bf28f42d-7eba-4ea2-904e-037aa7e098e4.csv


'bf28f42d-7eba-4ea2-904e-037aa7e098e4'

In [ ]:
!pip install --upgrade gspread

In [15]:
!date

Tue Aug 10 23:46:59 UTC 2021


## Train the mode (~ 10 minutes)

In [16]:
train_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"uncertainty": "False",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "True",
"split_uuid": "{}".format(split_uuid),
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [17]:
train_params = parse.wrapper(train_config)

In [18]:
train_model = mp.ModelPipeline(train_params)

## Train_model took ~ 18 minutes on a GPU  (~ 30 minutes on a CPU)

In [19]:
mp.ampl_version

'1.2.0'

In [20]:
dc.__version__

'2.6.0.dev'

In [21]:
%cd github
!ls

/content/github
AMPL  AMPL.build  AMPL.dist  __init___py.patch	transformations_py.patch


In [22]:
train_model.train_model()

2021-08-10 23:47:05,904 Attempting to load featurized dataset
2021-08-10 23:47:05,917 Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
2021-08-10 23:47:05,918 Featurized dataset not previously saved for dataset ChEMBL26_SCN5A_IC50_human_ml_ready, creating new
2021-08-10 23:47:05,931 Featurizing sample 0
2021-08-10 23:47:12,481 Featurizing sample 1000


number of features: 75


2021-08-10 23:47:18,984 Previous dataset split restored
2021-08-10 23:47:19,001 Wrote transformers to /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/a24bea03-41eb-4f0c-8686-96581d4f82c0/transformers.pkl
2021-08-10 23:47:19,003 Transforming response data
2021-08-10 23:47:19,726 Transforming response data
2021-08-10 23:47:19,857 Transforming response data


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2021-08-10 23:47:22,442 From /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/array_ops.py:5049: calling gather (from tensorflow.python.ops.array_ops) with validate_indices is deprecated and will be removed in a future version.
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
2021-08-10 23:47:39,299 Epoch 0: training r2_score = 0.009, validation r2_score = 0.057, test r2_score = -0.084
2021-08-10 23:47:40,066 Epoch 1: training r2_score = 0.023, validation r2_score = -0.060, test r2_score = -0.082
2021-08-10 23:47:40,711 Epoch 2: training r2_score = -0.190, validation r2_score = -0.367, test r2_score = -0.328
2021-08-10 23:47:41,355 Epoch 3: training r2_score = -0.079, validation r2_score = -0.306, test r2_score = -0.256
2021-08-10 23:47:41,989 Epoch 4: training r2_score = -0.180, validation r2_score = -0.336, test r2_score = -0.267
2021-08-10 23:47:42,617 Epoch 5: training r2_score = 0.

Wrote model tarball to /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready_model_a24bea03-41eb-4f0c-8686-96581d4f82c0.tar.gz


In [23]:
perf_df = cmp.get_filesystem_perf_results('/content/chembl_activity_models', pred_type='regression')
perf_df

Found data for 1 models under /content/chembl_activity_models


,model_uuid,ampl_version,model_type,dataset_key,featurizer,splitter,model_score_type,feature_transform_type,learning_rate,dropouts,layer_sizes,best_epoch,max_epochs,rf_estimators,rf_max_features,rf_max_depth,xgb_gamma,xgb_learning_rate,model_choice_score,train_r2_score,train_rms_score,train_mae_score,train_num_compounds,valid_r2_score,valid_rms_score,valid_mae_score,valid_num_compounds,test_r2_score,test_rms_score,test_mae_score,test_num_compounds
0,a24bea03-41eb-4f0c-8686-96581d4f82c0,1.2.0,NN,/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv,graphconv,scaffold,r2,normalization,0.0007,"0.00,0.00,0.00","64,64,32",22,100,NaN,NaN,NaN,NaN,NaN,0.296437,0.744052,0.324392,0.235616,1210,0.296437,0.49293,0.366969,259,0.284589,0.593709,0.40849,260


In [24]:
!date

Tue Aug 10 23:48:12 UTC 2021
